In [1]:
%pip install -r requirements.txt
%pip install requests

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sys
import requests
import time
import logging
import argparse
from typing import Optional

In [3]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Create directories
os.makedirs('utils', exist_ok=True)
os.makedirs('results/part_2', exist_ok=True)

## 1. Connecting to the Hugging Face API

In [4]:
import requests

url = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"
key = ""
hdrs = {"Authorization": f"Bearer {key}"}

def query(data):
    res = requests.post(url, headers=hdrs, json=data)
    print("code:", res.status_code)
    print("text:", res.text)
    try:
        return res.json()
    except:
        print("bad json")
        return {"err": "not json"}

msg = {"inputs": "What are the symptoms of diabetes?"}
out = query(msg)
print(out)

code: 401
text: {"error":"Invalid credentials in Authorization header"}
{'error': 'Invalid credentials in Authorization header'}


## 2. Creating Simple Chat Scripts

In [ ]:
# utils/one_off_chat.py
import requests
import argparse
import os

def get_response(prompt, model_name="HuggingFaceH4/zephyr-7b-beta", api_key=None):
    url = f"https://api-inference.huggingface.co/models/{model_name}"
    head = {"Authorization": f"Bearer {api_key}"} if api_key else {}

    data = {
    "inputs": prompt,
    "parameters": {
        "max_new_tokens": 50  
    }}
    try:
        res = requests.post(url, headers=head, json=data)
        res.raise_for_status()
        out = res.json()

        if isinstance(out, list) and "generated_text" in out[0]:
            return out[0]["generated_text"]
        else:
            return str(out)
    except Exception as e:
        return f"Error: {e}"

def run_chat(model_name, api_key):
    print("Simple LLM Chat — type 'exit' to quit")
    while True:
        msg = input("You: ")
        if msg.lower() == "exit":
            print("Bye!")
            break
        reply = get_response(msg, model_name=model_name, api_key=api_key)
        print("Bot:", reply)

def main():
    parser = argparse.ArgumentParser(description="Chat with an LLM")
    parser.add_argument("--model", default="HuggingFaceH4/zephyr-7b-beta")
    parser.add_argument("--api_key", default=os.getenv("HF_API_KEY"))
    args, _ = parser.parse_known_args()

    run_chat(model_name=args.model, api_key=args.api_key)

if __name__ == "__main__":
    main()

Simple LLM Chat — type 'exit' to quit
Bye!


### Contextual Conversation Script

In [11]:
# utils/conversation.py

import requests
import argparse
import os

def get_response(msg, hist=None, model="HuggingFaceH4/zephyr-7b-beta", key=None, keep=3):
    if hist is None:
        hist = []

    # build prompt from recent history
    lines = ""
    for q, a in hist[-keep:]:
        lines += f"User: {q}\nBot: {a}\n"
    lines += f"User: {msg}\nBot:"

    url = f"https://api-inference.huggingface.co/models/{model}"
    head = {"Authorization": f"Bearer {key}"} if key else {}
    data = {"inputs": lines}

    try:
        res = requests.post(url, headers=head, json=data)
        res.raise_for_status()
        out = res.json()
        if isinstance(out, list) and "generated_text" in out[0]:
            return out[0]["generated_text"].replace(lines, "").strip()
        else:
            return str(out)
    except Exception as e:
        return f"Error: {e}"

def run_chat(model, key):
    print("Chat w/ memory — type 'exit' to quit")
    hist = []
    while True:
        user = input("You: ")
        if user.lower() == "exit":
            print("Bye!")
            break
        bot = get_response(user, hist, model, key)
        hist.append((user, bot))
        print("Bot:", bot)

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model", default="HuggingFaceH4/zephyr-7b-beta")
    parser.add_argument("--api_key", default=os.getenv("HF_API_KEY"))
    args, _ = parser.parse_known_args()
    run_chat(args.model, args.api_key)

if __name__ == "__main__":
    main()

Chat w/ memory — type 'exit' to quit
Bye!


## 3. Testing and Evaluation

In [12]:
# utils/test_chat.py
import os
from one_off_chat import get_response as get_one_off_response

def test_chat(questions, model_name="HuggingFaceH4/zephyr-7b-beta", api_key=None):

    results = {}
    
    for question in questions:
        print(f"Testing question: {question}")
        response = get_one_off_response(question, model_name, api_key)
        results[question] = response
        
    return results

test_questions = [
    "What are the symptoms of gout?",
    "How is gout diagnosed?",
    "What treatments are available for gout?",
    "What lifestyle changes can help manage gout?",
    "What foods should be avoided with gout?"
]

def save_results(results, output_file="results/part_2/example.txt"):

    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("# LLM Chat Tool Test Results\n\n")
        f.write("## Usage Examples\n\n")
        f.write("```bash\n")
        f.write("# Run the one-off chat\n")
        f.write("python utils/one_off_chat.py\n\n")
        f.write("# Run the contextual chat\n")
        f.write("python utils/conversation.py\n")
        f.write("```\n\n")
        f.write("## Test Results\n\n")
        f.write("```csv\n")
        f.write("question,response\n")
        for question, response in results.items():
            q = question.replace(',', '').replace('\n', ' ')
            r = response.replace(',', '').replace('\n', ' ')
            f.write(f"{q},{r}\n")
        f.write("```\n")

# Run the test and save results
if __name__ == "__main__":
    model_name = "HuggingFaceH4/zephyr-7b-beta"
    api_key = os.getenv("HF_API_KEY")

    if not api_key:
        print("Please set HF_API_KEY environment variable.")
    else:
        results = test_chat(test_questions, model_name, api_key)
        save_results(results)
        print("Test results saved to results/part_2/example.txt")

ModuleNotFoundError: No module named 'one_off_chat'